<a href="https://colab.research.google.com/github/supriya759694/Code-CPP/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News

Importing The Dependency

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # Import the stopwords module
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer # Changed 'Tfidfvectorizer' to 'TfidfVectorizer'
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Changed 'LogisticRegration' to 'LogisticRegression'
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
news_dataset = pd.read_excel('/content/Book1.xlsx', engine='openpyxl')  # Read the Excel file
news_dataset.head()  # Display the first 5 rows of the DataFrame

,id,title,author,text,label,Column1,_1,_2,_3,_4,...,_671,_672,_673,_674,_675,_676,_677,_678,_679,_680
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Data Pre-processing

In [11]:
news_dataset.shape

(49160, 686)

In [ ]:
#counting number of missing values
news_dataset.isnull().sum()

,0
id,21
title,4491
author,8968
text,6092
label,6780
...,...
_676,49157
_677,49158
_678,49158
_679,49158


In [12]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+ ' '+news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
49155    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
49156    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
49157    Michael J. de la Merced and Rachel Abrams Macy...
49158    Alex Ansary NATO, Russia To Hold Parallel Exer...
49159              David Swanson What Keeps the F-35 Alive
Name: content, Length: 49160, dtype: object


In [ ]:
# separating the data & label
x = news_dataset.drop(columns='label',axis=1)
y = news_dataset['label']

In [ ]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
49155  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
49156  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
49157  20797  Macy’s Is Said to Receive Takeover Approach by...   
49158  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
49159  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

stemming is the process of reducing a word to its Root word

example: actor, actress, acting--> act


In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english') ]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)


In [ ]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
49155    jerom hudson rapper trump poster child white s...
49156    benjamin hoffman n f l playoff schedul matchup...
49157    michael j de la merc rachel abram maci said re...
49158    alex ansari nato russia hold parallel exercis ...
49159                            david swanson keep f aliv
Name: content, Length: 49160, dtype: object


In [ ]:
# separating the data and label
x = news_dataset['content'].values
y = news_dataset['label'].values

In [ ]:
print(x)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [ ]:
print(y)

['1' '0' '1' ... '0' '1' '1']


In [ ]:
y.shape

(49160,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x)

In [ ]:
print(x)

  (0, 297)	0.2697835881227123
  (0, 2693)	0.37106467958974654
  (0, 3207)	0.24921205124061444
  (0, 3904)	0.3646467301523997
  (0, 4112)	0.2710203402361038
  (0, 5401)	0.22244982828551804
  (0, 7651)	0.21812190235101173
  (0, 8427)	0.25021533496126735
  (0, 9431)	0.29207949345095163
  (0, 9740)	0.36881363085795166
  (0, 14713)	0.240220089426745
  (0, 17144)	0.28686841865194074
  (1, 1626)	0.2861123112064016
  (1, 2055)	0.15989421182106406
  (1, 2418)	0.37940920994382804
  (1, 3053)	0.19386386992049803
  (1, 3869)	0.26484118653858957
  (1, 5979)	0.7164444338387528
  (1, 7442)	0.19306868351873907
  (1, 18358)	0.30054956384952936
  (2, 3191)	0.32846506574265094
  (2, 3367)	0.47358930630843954
  (2, 5853)	0.39452892485901375
  (2, 6493)	0.33687969504730464
  (2, 10498)	0.47533653940864684
  :	:
  (49157, 3952)	0.21344021138248181
  (49157, 7690)	0.2207206301561853
  (49157, 9148)	0.22521497662840817
  (49157, 9821)	0.3661403173034771
  (49157, 10379)	0.29958467249993825
  (49157, 10461)	0.

Splitting the dataset to training & test data


In [ ]:
# Assuming 'y' is a Pandas Series or a NumPy array
# Find unique class labels and their counts
unique_classes, class_counts = np.unique(y, return_counts=True)

# Identify classes with only one sample
classes_with_one_sample = unique_classes[class_counts == 1]

# Filter out data points belonging to these classes
indices_to_keep = ~np.isin(y, classes_with_one_sample)  # keep elements that are NOT in classes_with_one_sample
x_filtered = x[indices_to_keep]
y_filtered = y[indices_to_keep]

# Now you can use train_test_split on the filtered dataset:
X_train, X_test, Y_train, Y_test = train_test_split(x_filtered, y_filtered, test_size=0.2, stratify=y_filtered, random_state=2)

Training the Model : Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation


accuracy score

In [ ]:
# accuracy score on the training data
x_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(x_train_prediction,Y_train)

In [ ]:
print('Accuracy score of the training data : ' , training_data_accuracy)

Accuracy score of the training data :  0.9497436159085691


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ' ,test_data_accuracy)

Accuracy score of the test data :  0.9348979591836735


Making a Predictive System

In [ ]:


X_new = X_test[3].reshape(1, -1)  # Ensure correct shape for the model

prediction = model.predict(X_new)  # Get prediction
# Convert prediction to numeric type before comparison
print(prediction)
predicted_class = (prediction.astype(int) > 0.5).astype(int)  # Adjust if needed

if predicted_class[0] == 0:
    print('The news is Real')
else:
    print('The news is Fake')


['0']
The news is Real


In [ ]:
print(Y_test[3])

0
